In [1]:
import nibabel
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [2]:
input_path = "/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT/"
output_path = "/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT-NIFTI/"

In [3]:
paths = os.listdir(input_path)
paths = [path for path in paths if path.endswith(".npz")]

In [5]:
paths

['val27365.npz',
 'val26902.npz',
 'val27050.npz',
 'val27049.npz',
 'val26904.npz',
 'val27047.npz',
 'val27046.npz',
 'val27044.npz',
 'val27043.npz',
 'val27162.npz',
 'val27025.npz',
 'val27164.npz',
 'val27026.npz',
 'val26886.npz',
 'val27168.npz',
 'val27170.npz',
 'val27172.npz',
 'val27027.npz',
 'val27006.npz',
 'val26905.npz',
 'val26915.npz',
 'val26789.npz',
 'val26917.npz',
 'val26890.npz',
 'val27051.npz',
 'val27174.npz',
 'val26885.npz',
 'val27178.npz',
 'val27182.npz',
 'val26918.npz',
 'val27041.npz',
 'val27022.npz',
 'val27038.npz',
 'val27161.npz',
 'val27160.npz',
 'val27009.npz',
 'val27037.npz',
 'val27036.npz',
 'val27011.npz',
 'val26919.npz',
 'val27035.npz',
 'val27159.npz',
 'val27158.npz',
 'val27157.npz',
 'val27153.npz',
 'val26884.npz',
 'val26882.npz',
 'val27152.npz',
 'val26895.npz',
 'val26880.npz',
 'val27151.npz',
 'val27149.npz',
 'val27028.npz',
 'val26897.npz',
 'val26920.npz',
 'val26784.npz',
 'val27030.npz',
 'val27147.npz',
 'val26899.npz

In [6]:
df_train = pd.read_csv("/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT/imgtrain_Abnormality_and_Location_Labels.csv")
df_valid = pd.read_csv("/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT/imgvalid_Abnormality_and_Location_Labels.csv")
df_test = pd.read_csv("/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT/imgtest_Abnormality_and_Location_Labels.csv")

df = pd.concat((df_train,df_valid,df_test)).reset_index(drop=True)

In [7]:
df

,NoteAcc_DEID,bandlike_or_linear*left_upper,bandlike_or_linear*left_mid,bandlike_or_linear*left_lower,bandlike_or_linear*right_upper,bandlike_or_linear*right_mid,bandlike_or_linear*right_lower,bandlike_or_linear*right_lung,bandlike_or_linear*left_lung,bandlike_or_linear*lung,...,other_path*esophagus,other_path*stomach,other_path*intestine,other_path*liver,other_path*gallbladder,other_path*kidney,other_path*adrenal_gland,other_path*spleen,other_path*pancreas,other_path*other_location
0,trn24737,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,trn08439,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,trn11613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,trn15638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,trn03087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625,tst29123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3626,tst34883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3627,tst34029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3628,tst34239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
labels = [
    'cardiomegaly',
    'pleural_effusion',
    "pneumonia",
    'pneumothorax',
    'atelectasis'
]

label_location = [i for i in list(df.columns)[1:] if i.split("*")[0] in labels]

label_list = list()

for i in range(len(df)):

    key_values = df.loc[i][df.loc[i]==1]
    valid = False

    for key in key_values.keys():
        if key in label_location:
            valid = True
    
    if valid:
        label_list.append(key_values)

In [9]:
label_ids = [i.name for i in label_list]

In [10]:
label_paths = [df.loc[i]["NoteAcc_DEID"]+ ".npz" for i in label_ids]

In [18]:
len(os.listdir( "/dtu/p1/johlau/Thesis-Synthex/data/RAD-ChestCT-DRR"))

634

In [19]:
len(label_paths)

1694

In [20]:
for path in tqdm(label_paths):
    ct = np.load(input_path + path)
    new_image = nibabel.Nifti1Image(ct["ct"], affine=np.eye(4))
    nibabel.save(new_image, output_path+path.split(".")[0] + ".nii.gz")

 39%|███▉      | 661/1694 [51:56<1:25:36,  4.97s/it]